# TensorBoard

[![Open in Colab](https://lab.aef.me/files/assets/colab-badge.svg)](https://colab.research.google.com/github/adamelliotfields/lab/blob/main/files/tf/tensorboard.ipynb)
[![Open in Kaggle](https://lab.aef.me/files/assets/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/adamelliotfields/lab/blob/main/files/tf/tensorboard.ipynb)
[![Render nbviewer](https://lab.aef.me/files/assets/nbviewer_badge.svg)](https://nbviewer.org/github/adamelliotfields/lab/blob/main/files/tf/tensorboard.ipynb)

Example logging metrics to TensorBoard.

**Notes**

Using the `LearningRateScheduler` callback logs to TensorBoard automatically; so this is just an example of a custom logger.

```python
import tensorflow as tf
from keras import callbacks, optimizers

class LearningRateLogger(callbacks.Callback):
    def __init__(self, log_dir):
        super().__init__()
        self.log_dir = log_dir
        self.file_writer = tf.summary.create_file_writer(log_dir)

    def on_epoch_end(self, epoch, logs=None):
        lr = self.model.optimizer.learning_rate
        if isinstance(lr, optimizers.schedules.LearningRateSchedule):
            lr = lr(epoch)
        else:
            lr = lr.numpy()  # tensor to scalar
        with self.file_writer.as_default():
            tf.summary.scalar("learning_rate", data=lr, step=epoch)
```

In [1]:
%load_ext tensorboard

In [2]:
import math
import os
import shutil
import tensorflow as tf

from datetime import datetime
from keras import Input, Model, callbacks, initializers, layers, optimizers, random
from keras_tuner import RandomSearch

In [3]:
# xor returns 1 if inputs are not equal and 0 otherwise
X = tf.constant([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=tf.float32)
y = tf.constant([[0], [1], [1], [0]], dtype=tf.float32)

# rng seed
seed = random.SeedGenerator(seed=42)

# weight initialization
initializer_he = initializers.HeNormal(seed=seed)


# custom learning rate schedule
def lr_schedule(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * math.exp(-0.1)


def build_model(hp):
    activation = hp.Choice("activation", ["tanh", "gelu", "mish"])

    optimizer = optimizers.Lion(learning_rate=0.01)
    inputs = Input(shape=(2,))
    x = inputs

    for _ in range(hp.Int("layers", 2, 3)):
        x = layers.Dense(
            32,
            activation=activation,
            kernel_initializer=initializer_he,
        )(x)

    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [4]:
if os.path.exists("logs"):
    shutil.rmtree("logs")

now = datetime.now().strftime("%Y%m%d-%H%M%S")

# we only have 6 possible combinations
tuner = RandomSearch(
    build_model,
    max_trials=6,
    objective="loss",
    directory="logs",
    project_name=f"{now}/tuner",
)
tuner.search(
    X,
    y,
    epochs=100,
    batch_size=4,
    verbose=0,
    callbacks=[
        callbacks.TensorBoard(log_dir=f"logs/{now}"),
        callbacks.LearningRateScheduler(lr_schedule),
        callbacks.EarlyStopping(monitor="loss", min_delta=0.001, patience=20),
    ],
)